<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=150 align="right">
<img src=images/Acme.jpeg width=100 align="left">


# UC-4 App Role Authentication
---
## Business Values
 * Provide a way for operators to manage policy without being involved in generating tokens or creds for applications (minimize coordination) 
 * Operators have less work to do
 * Orchestration tools can take over the task of giving applications secrets access

## How AppRole Auth Method works

Vault provides internal and external authentication methods.  External methods are called *tusted third-party authenticators* such as AWS, LDAP, Github, etc.

In some situations trusted third-party authenticator is not available, so Vault has an alternate approach - **AppRole**
* AppRole generates **RoleID** and **SecretID**
    - RoleID is an identifier similar to username
    - SecretID is a credential to login (similart to password)  

<br />   
<img src="images/vault-approle-workflow.png" width=1000>
<img src="images/vault-approle-pull.png" width=1000>
<img src="images/vault-approle-push.png" width=1000>

### Setup
---

In [ ]:
export VAULT_ADDR=http://127.0.0.1:8200
export VAULT_TOKEN=root
export VAULT_SKIP_VERIFY=true

In [ ]:
unset VAULT_TOKEN; vault login root

In [ ]:
vault status

In [ ]:
vault -version

### Demo
---

### Enable the approle authentication method


In [ ]:
vault auth enable approle

### Create a secret
NOTE: num_uses determines how many times secret can be used.

In this example, a SecretID of the myapp role can be used for up to 5 times to authenticate and fetch a client token. After the number of uses is reached, the SecretID expires and you would need to generate a new one. This is similar to forcing a password rotation.

In [ ]:
vault write auth/approle/role/myapp secret_id_num_uses=3 secret_id_ttl=5m

##### Read the secret

In [ ]:
vault read auth/approle/role/myapp

### Create a role id 
Typically a constant value, provide this to the app developer team

In [ ]:
vault read auth/approle/role/myapp/role-id

**Attempt to read the role id (same as above) using the API**. 


In [ ]:
export APPROLE="myapp"
export VAULT_TOKEN=root
ROLE_ID=$(curl -s -H "X-Vault-Token: ${VAULT_TOKEN}" "http://127.0.0.1:8200/v1/auth/approle/role/${APPROLE}/role-id" | jq -r '.data.role_id')
echo $ROLE_ID

### Generate a Secret ID  

In [ ]:
vault write -f auth/approle/role/myapp/secret-id

In [ ]:
#another secret-id
vault write -f auth/approle/role/myapp/secret-id

**Using CURL instead**  

In [ ]:
SECRET_ID=$(curl -s -X POST -H "X-Vault-Token:${VAULT_TOKEN}" "http://127.0.0.1:8200/v1/auth/approle/role/${APPROLE}/secret-id" | jq -r '.data.secret_id')
echo ${SECRET_ID}

### Get a token for logging into Vault using roleID and secretID

In [ ]:
vault write auth/approle/login role_id=${ROLE_ID} secret_id=${SECRET_ID}


**Again, we can do the same using CURL**

In [ ]:
APP_ROLE_TOKEN=$(curl -s -X POST -d '{"role_id": "'"${ROLE_ID}"'", "secret_id": "'"${SECRET_ID}"'"}' http://127.0.0.1:8200/v1/auth/approle/login | jq -r '.auth.client_token')
echo ${APP_ROLE_TOKEN}

In [ ]:
vault login ${APP_ROLE_TOKEN}

## Response Wrap the SecretID
The RoleID is equivalent to a username, and SecretID is the corresponding password. The app needs both to log in with Vault. Naturally, the next question becomes how to deliver those values to the client securely.

A common solution involves three personas instead of two: admin, app, and trusted entity. The trusted entity delivers the RoleID and SecretID to the client by separate means.

For example, Terraform as a trusted entity can deliver the RoleID onto the virtual machine. When the app runs on the virtual machine, the RoleID already exists on the virtual machine.

<img src="images/vault-approle-workflow2.png">

SecretID is like password.  Use **response wrapping** so that only expecting client can unwrap the SecretID.

In [ ]:
## parameter order matters
vault write -wrap-ttl=60s -f auth/approle/role/myapp/secret-id.

In [ ]:
VAULT_TOKEN=s.ILPSggHj70JSzTWpcWtot7nU vault unwrap

In [ ]:
# should give error
VAULT_TOKEN=s.ILPSggHj70JSzTWpcWtot7nU vault unwrap

&nbsp;

---
#### Thank you.
<img src=images/HashiCorp_PrimaryLogo_Black_RGB.png width=100 align="left">

---
# AppRole Auhentication Method

Vault provides internal and external authentication methods.  External methods are called *tusted third-party authenticators* such as AWS, LDAP, Github, etc.

In some situations trusted third-party authenticator is not available, so Vault has an alternate approach - **AppRole** .   
AppRole allows machines or apps to authenticate.  This auth method is oriented to automated workflows (machines and services) and is less useful for human operators.

* AppRole generates **RoleID** and **SecretID**
    - RoleID is an identifier similar to username
    - SecretID is a credential to login (similart to password)   
    

### Setup
<img src="images/vault-approle-workflow.png" width=800>


In [ ]:
vault login root
vault auth enable approle

In [ ]:
vault write auth/approle/role/tio-crm \
  token_num_users=3 \
  token_ttl=10m \
  token_max_ttl=30m \
  secret_id_ttl=5m \
  secret_id_num_uses=40 

### RoleID
RoleID is a unique identifier for the app.  They can be set to particular values to match introspected information by the client (ex: client's domain name).

In [ ]:
vault read auth/approle/role/tio-crm/role-id

### SecretID
SecretID is like a password, it is intended to always be secret.  SecretIDs can be created agains an AppRole either via generation of a 128-bit purely random UUID by the role itself or via specific custom values.  SecretIDs have properties like usage-limit, TTLs, and expirations.

**Pull and Push SecretID Modes**  
If SecretID is fetched from an AppRole, this is Pull Mode.  Most cases Pull mode is the better approach.

<img src="images/vault-approle-pull.png" width=800>  

If a "custom" secretID is set by the client, it is Push Mode.  
<img src="images/vault-approle-push.png" width=800>

Push Mode requires some other system to have knowledge of the full set of credentials (RoleID and SecretID) in order to create the entry.   
However in Pull Mode, the SecretID can be kept confidential from all parties except for the final authenticating client by using Response w Wrapping.
  


In [ ]:
vault write -f auth/approle/role/tio-crm/secret-id

### Login

In [ ]:
vault write auth/approle/login \
  role_id=d56c9c46-7198-4238-fb0c-3a2b26e97e18   \
  secret_id=e151cb41-0152-8974-e259-75d9fd7c2972 

**Using cURL**  
The default endpoint is `auth/approle/login`.  The response will contain the token at `auth.client_token`

In [ ]:
curl -s --request POST \
    --data "{\"role_id\":\"d56c9c46-7198-4238-fb0c-3a2b26e97e18\", \"secret_id\":\"e151cb41-0152-8974-e259-75d9fd7c2972\"}" \
    http://127.0.0.1:8200/v1/auth/approle/login |jq